<a href="https://colab.research.google.com/github/CamiDRF/PruebaTecnicaFinaipro/blob/prueba-tecnica/Prueba_T%C3%A9cnica_Ingeniero_de_Datos_(Semi_Senior)_Finaipro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**1. Ejercicio de SQL (40 minutos)**
**Contexto**: Construir las siguientes 3 tablas en una base de datos SQL (puede ser PostgreSQL, MySQL, etc.), y responder a las preguntas planteadas.

**Tablas**:
- `orders` (order_id, customer_id, order_date, total_amount)
- `customers` (customer_id, name, email, registration_date)
- `products` (product_id, name, price, category)

##Creacion de Base de Datos e Insersion de datos ficticios



In [ ]:
# Llammado de biblioteca SQLite
import pandas as pd
import sqlite3 #Importar motor de bases de datos

In [ ]:
# Creacion de base de datos
rutadb = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/dbPrueba.sqlite"

In [ ]:
# Creacion de tablas en ruta del proyecto
queryOrders = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

queryCustomers = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    email TEXT,
    registration_date DATE
);
"""

queryProducts = """
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY,
    name TEXT,
    price DECIMAL(10, 2),
    category TEXT
);
"""

# Conectar a la base de datos
con = sqlite3.connect(rutadb)
cursor = con.cursor()

# Ejecutar las consultas
cursor.executescript(queryCustomers)
cursor.executescript(queryProducts)
cursor.executescript(queryOrders)

# Confirmar cambios y cerrar conexión
con.commit()
con.close()

In [ ]:
# Insersion de datos de prueba: Para efectos practicos se generaron aleatoriamente registros en https://mockaroo.com/

# Conexiona la BD
con = sqlite3.connect(rutadb)

# Función para ejecutar archivos SQL
def ejecutar_sql(ruta_sql):
    with open(ruta_sql, "r", encoding="utf-8") as archivo:
        sql_script = archivo.read()
    con.executescript(sql_script)

# Rutas de los archivos SQL en Google Drive
ruta_customers = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/customers.sql"
ruta_orders = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/orders.sql"
ruta_products = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/products.sql"

# Ejecutar cada archivo SQL en la base dbPrueba.sqlite
ejecutar_sql(ruta_customers)
ejecutar_sql(ruta_orders)
ejecutar_sql(ruta_products)

# Cerrar la conexión
con.close()

## Pregutan 1: Obtener los 10 clientes con más órdenes en los últimos 6 meses, mostrando nombre, email y cantidad de órdenes.

In [ ]:
# Conectar a la BD
con = sqlite3.connect("/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/dbPrueba.sqlite")

# Ejecutar la consulta y guardar los resultados en un DataFrame
query = """
SELECT c.name, c.email, COUNT(o.order_id) AS total_orders
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
WHERE o.order_date >= '2023-11-01' -- Se quiso usar >= DATE('now', '-6 months') pero no hay suficientes registros
GROUP BY c.customer_id
ORDER BY total_orders DESC
LIMIT 10;
"""

df = pd.read_sql(query, con)

# Mostrar el resultado
print(df)

# Cerrar la conexión **después** de leer los datos
con.close()

               name                         email  total_orders
0  Carlos Rodríguez  carlos.rodriguez@example.com             4
1       Javier Díaz       javier.diaz@example.com             2
2         Ana López         ana.lopez@example.com             2
3     Diego Ramírez     diego.ramirez@example.com             1
4    Camila Mendoza    camila.mendoza@example.com             1
5  Fernando Álvarez  fernando.alvarez@example.com             1
6     Valeria Gómez     valeria.gomez@example.com             1
7     Sofía Herrera     sofia.herrera@example.com             1
8     Miguel Torres     miguel.torres@example.com             1
9    Lucía Martínez    lucia.martinez@example.com             1


## 2. Calcular el promedio de gasto por categoría de producto para clientes registrados en un año en particular.

In [ ]:
# Se identifico que es necesaria una tabla intermedia, para conectar ordenes con productos

# Conectar a SQLite
con = sqlite3.connect(rutadb)
cursor = con.cursor()

# Crear la tabla order_items con claves foráneas a orders y products
cursor.executescript("""
CREATE TABLE IF NOT EXISTS order_items (
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
""")

con.commit()
con.close()

In [ ]:
# Se insertan datos de prueba para el ejerccio

con = sqlite3.connect(rutadb)
cursor = con.cursor()

# Insertar productos en órdenes
cursor.executescript("""
INSERT INTO order_items (order_id, product_id, quantity) VALUES (1, 1, 1);
INSERT INTO order_items (order_id, product_id, quantity) VALUES (1, 2, 1);
INSERT INTO order_items (order_id, product_id, quantity) VALUES (2, 2, 1);
INSERT INTO order_items (order_id, product_id, quantity) VALUES (3, 2, 1);
""")

con.commit()
con.close()

In [ ]:
# ya con la relacion creada y los datos ficticios se crea la consulta

# Conectar a la base de datos
con = sqlite3.connect(rutadb)

# Consulta SQL: Promedio de gasto por categoría de producto en clientes registrados en 2023
query_avg_spent = """
SELECT p.category, AVG(oi.quantity * p.price) AS avg_spent
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE STRFTIME('%Y', c.registration_date) = '2023'
GROUP BY p.category
ORDER BY avg_spent DESC;
"""

# Ejecutar la consulta y guardar el resultado en un DataFrame
df_avg_spent = pd.read_sql(query_avg_spent, con)

# Mostrar el resultado
display(df_avg_spent)

# Cerrar la conexión
con.close()

,category,avg_spent
0,Electrónica,1050.245


## 3. Escribir una consulta para detectar registros duplicados o inconsistentes en la tabla `customers`

In [ ]:
import sqlite3
import pandas as pd

rutadb = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/dbPrueba.sqlite"

con = sqlite3.connect(rutadb)

query_check_customers = """
WITH duplicated_customers AS (
    SELECT customer_id, name, email, registration_date
    FROM customers
    GROUP BY customer_id, name, email, registration_date
    HAVING COUNT(*) > 1
),
invalid_emails AS (
    SELECT customer_id, name, email, registration_date
    FROM customers
    WHERE email NOT LIKE '%@%' OR email LIKE '%..%' OR email LIKE '% %'
),
null_dates AS (
    SELECT customer_id, name, email, registration_date
    FROM customers
    WHERE registration_date IS NULL
)
SELECT * FROM duplicated_customers
UNION ALL
SELECT * FROM invalid_emails
UNION ALL
SELECT * FROM null_dates;
"""

df_check_customers = pd.read_sql(query_check_customers, con)

display(df_check_customers)

con.close()

,customer_id,name,email,registration_date


#**2. Ejercicio de Python (ETL) (60 minutos)**
**Contexto**: Procesar un archivo CSV (ej: `sales_data.csv`) con registros de ventas y transformarlo para cargarlo en una base de datos.

**Requisitos**:
1. **ETL**: Implementar un proceso ETL sobre el archivo CSV. La carga del fichero la simularemos con un archivo de salida en formato parquet.

**Formato de entrega**: Script de Python (`etl.py`) + archivo de salida.

## Preparación de la informacion (cargue de librerias y de data)

In [ ]:
# Instalacion de Pyarrow
!pip install pandas pyarrow

In [ ]:
import pandas as pd

# Definicion el nombre del archivo fuente CSV y Parquet de salida
csv_file = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/sales_data.csv"  # Ajusta la ruta si es diferente
parquet_file = "/content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/sales_data.parquet"



## Analisis Exploratorio de la fuente

In [ ]:
# Extrar: Cargar el archivo CSV
df = pd.read_csv(csv_file)

 #Muestra la informacion resumuda que contiene cada columna y los tipos de datos
df.info()

# Identificar los registros nulos
df_nulls = df[df.isnull().any(axis=1)]
display(df_nulls)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   order_id     100 non-null    int64  
 1   customer_id  99 non-null     object 
 2   product_id   100 non-null    object 
 3   order_date   100 non-null    object 
 4   quantity     90 non-null     float64
 5   price        96 non-null     float64
 6   region       100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


,order_id,customer_id,product_id,order_date,quantity,price,region
2,1003,NaN,P010,2023-05-16,NaN,90.75,East
6,1007,C007,P025,2023-05-20,4.0,NaN,East
14,1015,C015,P050,2023-05-28,NaN,85.40,East
23,1024,C024,P080,2023-06-06,NaN,65.90,West
32,1033,C003,P010,2023-06-15,NaN,90.75,North
36,1037,C007,P025,2023-06-19,4.0,NaN,North
44,1045,C015,P050,2023-06-27,NaN,85.40,North
53,1054,C024,P080,2023-07-06,NaN,65.90,South
62,1063,C003,P010,2023-07-15,NaN,90.75,East
66,1067,C007,P025,2023-07-19,4.0,NaN,East


In [ ]:
# Se identifica que para la variable de precio y cantidad existen valores nulos
# Se desea saber si es posible utilizar estadisitica para reemplzar los valores
# esto debido a que represetan el 10% de la totalidad de los registros

std_price = df['price'].std()
mean_price = df['price'].mean()
median_price = df['price'].median()

print(f"📊 Desviación estándar: {std_price:.2f}")
print(f"✅ Media: {mean_price:.2f}")
print(f"✅ Mediana: {median_price:.2f}")


📊 Desviación estándar: 51.06
✅ Media: 106.26
✅ Mediana: 95.20


## Transformacion y Limpieza de datos

Dada la desviacion estandar, y la cantidad de registros se descarta algortimos como KNN para completar los nulos, tambien se descarta el uso extrapolacion de datos, dado que los registros, con datos nulos son del mismo producto dado que los valores nulos solo representan el 14% de la poblacion y por mantener la canlidad de los datos, se opta por eliminarlos

In [ ]:
# Elimancion de los registros con nulos
df.dropna(subset=['price', 'quantity'], inplace=True)

# Reemplazo de - por / en order_date
df['order_date'] = df['order_date'].str.replace("-", "/")

# Cmabiar tipo de dato a fecha
df['order_date'] = pd.to_datetime(df['order_date'], format='mixed', errors='coerce')

# Cambiar region a tipo categoria
df['region'] = df['region'].astype('category')

# Cambiar tipo de cantidd a entero
df['quantity'] = df['quantity'].astype(int)


#Cambiar tipo de custormer_id a string y product_id a string
df['customer_id'] = df['customer_id'].astype(str).str[:15]
df['product_id'] = df['product_id'].astype(str).str[:15]
df.info()
display(df)


<class 'pandas.core.frame.DataFrame'>
Index: 86 entries, 0 to 99
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     86 non-null     int64         
 1   customer_id  86 non-null     object        
 2   product_id   86 non-null     object        
 3   order_date   86 non-null     datetime64[ns]
 4   quantity     86 non-null     int64         
 5   price        86 non-null     float64       
 6   region       86 non-null     category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 5.0+ KB


,order_id,customer_id,product_id,order_date,quantity,price,region
0,1001,C001,P001,2023-05-15,2,150.50,North
1,1002,C002,P005,2023-05-15,1,200.00,South
3,1004,C004,P015,2023-05-17,3,120.00,West
4,1005,C005,P001,2023-05-18,1,150.50,North
5,1006,C006,P020,2023-05-19,2,75.30,South
...,...,...,...,...,...,...,...
94,1095,C005,P001,2023-08-16,1,150.50,East
95,1096,C006,P020,2023-08-17,2,75.30,West
97,1098,C008,P001,2023-08-19,1,150.50,South
98,1099,C009,P030,2023-08-20,2,60.25,East


## Transformacion de archivo a tipo Parquet

In [ ]:
#Guardar el archivo en formato Parquet
df.to_parquet(parquet_file, engine='pyarrow')

print("Proceso ETL completado. Archivo Parquet generado:", parquet_file)

Proceso ETL completado. Archivo Parquet generado: /content/drive/MyDrive/Prueba Técnica: Ingeniero de Datos (Semi-Senior) - Finaipro/sales_data.parquet
